In [1]:
# Install the required packages
# !pip install -q yfinance==0.2.44 # Version 0.2.45 (lastest as of 2024-10-21) has an import error.
# !pip install -q pandas
# !pip install -q numpy
# !pip install -q scipy
# !pip install -q matplotlib
# !pip install -q holoviews

In [2]:
# Import in-house modules from the 'utilities' package
from utilities import calculate_bollinger_bands, calculate_rsi, calculate_daily_volatility
from utilities import print_title, print_label, print_footer
from utilities import generate_trading_signals
from utilities import fetch_and_download_sp500_data, sp500_data_for_today
from utilities import print_dataframe_report, save_data
from utilities import temporal_train_test_split

In [3]:
# Import libraries for data analysis and visualization
import pandas as pd

# Import libraries for signal processing and peak detection
from scipy.signal import find_peaks
import numpy as np

# Import libraries for plotting and visualization
import matplotlib.pyplot as plt
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

# Import libraries for data storage and retrieval
from pathlib import Path

In [4]:
# Set display options for Pandas (optional, but often helpful)
pd.set_option('display.float_format', lambda x: '%.3f' % x) # Suppress scientific notation

### Data Collection
___

In [5]:
# Syntax: fetch_and_download_sp500_data(start_date="YYYY-MM-DD", end_date="YYYY-MM-DD")
# To add an end date, put end_date="YYYY-MM-DD"

historical_data = fetch_and_download_sp500_data(start_date="2007-01-01")

display(historical_data.head(2))
display(historical_data.tail(2))

[*********************100%***********************]  501 of 501 completed


╔═══════════════════════════════════════════════════════════════╗
║            Download Report for S&P 500 `adj close`            ║
╠═══════════════════════════════════════════════════════════════╣
║ Total Requested Tickers:       |             501              ║
║ Total Downloaded Tickers:      |             501              ║
║ Requested Date Range:          |   2007-01-01 to 2024-10-27   ║
║ Downloaded Date Range:         |   2007-01-03 to 2024-10-25   ║
║                       S&P 500 Data Downloaded Successfully... ║
╚═══════════════════════════════════════════════════════════════╝


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-03 00:00:00+00:00,21.975,2.527,0.000,0.000,15.535,7.496,26.394,39.920,21.268,20.791,...,71.602,14.521,55.839,12.268,40.008,0.000,14.979,67.253,34.880,0.000
2007-01-04 00:00:00+00:00,22.046,2.583,0.000,0.000,15.830,7.492,26.988,40.820,21.637,20.674,...,71.779,14.375,55.862,12.278,39.258,0.000,14.938,68.753,34.680,0.000


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-24 00:00:00+00:00,130.690,230.570,189.650,132.750,116.540,107.040,364.230,482.870,229.680,56.500,...,290.530,32.130,97.900,65.110,119.590,130.650,133.160,104.000,362.050,181.500
2024-10-25 00:00:00+00:00,130.190,231.410,187.850,134.580,114.220,105.300,360.800,483.720,230.170,56.560,...,290.450,31.800,97.990,64.460,119.490,130.420,133.040,102.350,360.090,180.010


In [ ]:
# Syntax: sp500_data_for_today(time="15:59:00"):
# The default time is 15:59:00, which is the last minute of the trading day (3:59 PM)
# The time must be in the format "HH:MM:SS"

# todays_data = sp500_data_for_today(time="15:59:00")

In [7]:
# data = pd.concat([historical_data, todays_data], axis=0)

# print("Shape:", data.shape)
# display(data.head(10))
# display(data.tail(10))

data = historical_data.copy()

In [8]:
# Inspect for missing values
print("Missing Values:", data.isnull().sum().sum())

Missing Values: 0


### Feature Engineering
___

In [21]:
def generate_directions(data):
    next_day = data.shift(-1)  # Next day's price data
    prev_day = data.shift(1)   # Previous day's price data

    today_to_tomorrow = np.sign(next_day - data)
    yesterday_to_today = np.sign(data - prev_day)

    return today_to_tomorrow, yesterday_to_today

In [22]:
upper_band, lower_band = calculate_bollinger_bands(data)

today_to_tomorrow, yesterday_to_today = generate_directions(data)

# Create a dictionary of feature dataframes
feature_dataframes = {
    # Shifted Adjusted Close Prices
    "next_day_adj_close": data.shift(-1).copy(),
    "prev_day_adj_close": data.shift(1).copy(),

    # Directional Indicators
    "today_to_tomorrow": today_to_tomorrow.copy(),
    "yesterday_to_today": yesterday_to_today.copy(),

    # Returns and Volatility Calculations
    "returns": data.pct_change().copy(),
    "daily_volatility": data.apply(calculate_daily_volatility).copy(),

    # Technical Indicators
    "rsi": data.apply(calculate_rsi).copy(),
    "sma_50": data.rolling(window=50).mean().copy(),
    "sma_100": data.rolling(window=100).mean().copy(),
    "sma_200": data.rolling(window=200).mean().copy(),
    "upper_band": upper_band.copy(),
    "lower_band": lower_band.copy(),

    # Support and Resistance Levels
    "support": data.rolling(window=50).min().copy(),
    "resistance": data.rolling(window=50).max().copy(),

    # Trading Signals
    "actions": data.apply(generate_trading_signals).copy()
}

In [24]:
# Inspect DataFrames
select_df = "today_to_tomorrow"

inspect_df = feature_dataframes[select_df].copy()

print_dataframe_report(inspect_df, select_df)
display(inspect_df.head(5))
display(inspect_df.tail(5))

╔═══════════════════════════════════════════════════════════════╗
║             `Today_to_tomorrow` DataFrame Report              ║
╠═══════════════════════════════════════════════════════════════╣
║ Index Range:                   |   2007-01-03 to 2024-10-25   ║
║ `Today_to_tomorrow` Data Ty... |      [dtype('float64')]      ║
║ `Today_to_tomorrow` DF Shape:  |         (4485, 501)          ║
║ Columns with null values:      |             501              ║
║ Rows with null values:         |              1               ║
║ Total Unique Values:           |             1503             ║
╚═══════════════════════════════════════════════════════════════╝


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-03 00:00:00+00:00,1.000,1.000,0.000,0.000,1.000,-1.000,1.000,1.000,1.000,-1.000,...,1.000,-1.000,1.000,1.000,-1.000,0.000,-1.000,1.000,-1.000,0.000
2007-01-04 00:00:00+00:00,-1.000,-1.000,0.000,0.000,0.000,-1.000,-1.000,-1.000,-1.000,-1.000,...,-1.000,-1.000,1.000,-1.000,1.000,0.000,-1.000,-1.000,-1.000,0.000
2007-01-05 00:00:00+00:00,-1.000,1.000,0.000,0.000,1.000,1.000,1.000,-1.000,1.000,1.000,...,-1.000,1.000,1.000,0.000,-1.000,0.000,-1.000,-1.000,1.000,0.000
2007-01-08 00:00:00+00:00,1.000,1.000,0.000,0.000,1.000,-1.000,-1.000,-1.000,-1.000,-1.000,...,-1.000,-1.000,1.000,-1.000,-1.000,0.000,1.000,1.000,-1.000,0.000
2007-01-09 00:00:00+00:00,-1.000,1.000,0.000,0.000,1.000,1.000,1.000,-1.000,1.000,-1.000,...,-1.000,-1.000,1.000,1.000,-1.000,0.000,1.000,-1.000,1.000,0.000


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-21 00:00:00+00:00,-1.000,-1.000,1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,...,1.000,-1.000,1.000,1.000,1.000,-1.000,-1.000,-1.000,-1.000,1.000
2024-10-22 00:00:00+00:00,-1.000,-1.000,-1.000,-1.000,1.000,-1.000,-1.000,-1.000,1.000,1.000,...,1.000,1.000,-1.000,1.000,-1.000,1.000,1.000,1.000,-1.000,-1.000
2024-10-23 00:00:00+00:00,-1.000,-1.000,1.000,1.000,-1.000,1.000,-1.000,-1.000,1.000,1.000,...,-1.000,1.000,1.000,1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000
2024-10-24 00:00:00+00:00,-1.000,1.000,-1.000,1.000,-1.000,-1.000,-1.000,1.000,1.000,1.000,...,-1.000,-1.000,1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000
2024-10-25 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Preprocessing
___

In [25]:
# Initialize an empty dictionary for clean dataframes
clean_feature_dataframes = {}

for key, df in feature_dataframes.items():
    
    # Perform forward and backward filling, and then fill remaining NaNs with 0 or mean
    filled_df = df.copy()
    # filled_df.ffill(inplace=True)
    # filled_df.bfill(inplace=True)
    
    # Store the cleaned dataframe in the new dictionary
    clean_feature_dataframes[key] = filled_df

In [26]:
# Inspect DataFrames
select_df = "prev_day_adj_close"

inspect_df = clean_feature_dataframes[select_df]

print_dataframe_report(inspect_df, select_df)
display(inspect_df.head(5))
display(inspect_df.tail(5))

╔═══════════════════════════════════════════════════════════════╗
║             `Prev_day_adj_close` DataFrame Report             ║
╠═══════════════════════════════════════════════════════════════╣
║ Index Range:                   |   2007-01-03 to 2024-10-25   ║
║ `Prev_day_adj_close` Data T... |      [dtype('float64')]      ║
║ `Prev_day_adj_close` DF Shape: |         (4485, 501)          ║
║ Columns with null values:      |             501              ║
║ Rows with null values:         |              1               ║
║ Total Unique Values:           |           1913609            ║
╚═══════════════════════════════════════════════════════════════╝


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-03 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-04 00:00:00+00:00,21.975,2.527,0.000,0.000,15.535,7.496,26.394,39.920,21.268,20.791,...,71.602,14.521,55.839,12.268,40.008,0.000,14.979,67.253,34.880,0.000
2007-01-05 00:00:00+00:00,22.046,2.583,0.000,0.000,15.830,7.492,26.988,40.820,21.637,20.674,...,71.779,14.375,55.862,12.278,39.258,0.000,14.938,68.753,34.680,0.000
2007-01-08 00:00:00+00:00,21.840,2.565,0.000,0.000,15.830,7.419,26.611,40.620,21.391,20.642,...,71.033,14.329,56.193,12.032,39.538,0.000,14.808,68.709,34.330,0.000
2007-01-09 00:00:00+00:00,21.764,2.578,0.000,0.000,15.887,7.462,26.996,40.450,21.559,20.745,...,70.553,14.455,57.254,12.032,39.220,0.000,14.773,68.264,34.400,0.000


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-21 00:00:00+00:00,138.830,235.000,188.860,136.450,119.390,108.530,376.860,494.900,228.330,56.400,...,290.030,33.060,99.440,64.070,120.010,134.320,134.270,106.310,375.450,193.280
2024-10-22 00:00:00+00:00,136.760,236.480,186.540,137.190,116.990,108.500,376.030,497.710,225.670,56.130,...,290.500,32.290,98.370,63.430,120.080,132.520,134.000,105.200,372.590,189.450
2024-10-23 00:00:00+00:00,133.460,235.860,188.760,135.590,116.120,107.790,371.790,493.110,224.620,55.690,...,291.740,31.650,99.400,63.680,120.700,131.370,133.360,104.260,372.000,189.510
2024-10-24 00:00:00+00:00,133.030,230.760,187.880,131.920,117.010,107.000,370.630,485.030,226.850,56.080,...,291.920,31.850,97.680,64.320,120.270,131.720,134.020,104.700,368.090,188.990
2024-10-25 00:00:00+00:00,130.690,230.570,189.650,132.750,116.540,107.040,364.230,482.870,229.680,56.500,...,290.530,32.130,97.900,65.110,119.590,130.650,133.160,104.000,362.050,181.500


### Exploratory Data Analysis (EDA):
___

In [27]:
# Check for missing values in the cleaned dataframes
for name, df in clean_feature_dataframes.items():
    print(f"`{name}` DataFrame Missing Values:", df.isnull().sum().sum())

`next_day_adj_close` DataFrame Missing Values: 501
`prev_day_adj_close` DataFrame Missing Values: 501
`today_to_tomorrow` DataFrame Missing Values: 501
`yesterday_to_today` DataFrame Missing Values: 501
`returns` DataFrame Missing Values: 162562
`daily_volatility` DataFrame Missing Values: 172674
`rsi` DataFrame Missing Values: 162925
`sma_50` DataFrame Missing Values: 24549
`sma_100` DataFrame Missing Values: 49599
`sma_200` DataFrame Missing Values: 99699
`upper_band` DataFrame Missing Values: 9519
`lower_band` DataFrame Missing Values: 9519
`support` DataFrame Missing Values: 24549
`resistance` DataFrame Missing Values: 24549
`actions` DataFrame Missing Values: 164040


In [28]:
# for name, df in clean_feature_dataframes.items():
#     print_title(f"`{name}` DataFrame", closed_corners=False)
#     print_label("Shape:", f"{df.shape}")
#     print_label("Length:", f"{len(df)}", closed_corners=True)

#     display(df.head())

### Merging S&P Data
___

In [29]:
# Assign clean dataframes to individual variables for easier access

# Shifted Adjusted Close Prices
next_day_adj_close = clean_feature_dataframes["next_day_adj_close"]
prev_day_adj_close = clean_feature_dataframes["prev_day_adj_close"]

# Directional Indicators
today_to_tomorrow = clean_feature_dataframes["today_to_tomorrow"]
yesterday_to_today = clean_feature_dataframes["yesterday_to_today"]

# Returns and Volatility Calculations
returns = clean_feature_dataframes["returns"]
daily_volatility = clean_feature_dataframes["daily_volatility"]

# Technical Indicators
rsi = clean_feature_dataframes["rsi"]
sma_50 = clean_feature_dataframes["sma_50"]
sma_100 = clean_feature_dataframes["sma_100"]
sma_200 = clean_feature_dataframes["sma_200"]
upper_band = clean_feature_dataframes["upper_band"]
lower_band = clean_feature_dataframes["lower_band"]

# Support and Resistance Levels
support = clean_feature_dataframes["support"]
resistance = clean_feature_dataframes["resistance"]

# Trading Signals
actions = clean_feature_dataframes["actions"]

In [30]:
# Compile indicators into a single DataFrame
length = len(data) * len(data.columns)

indicators_df = pd.DataFrame({
    'Date': np.repeat(data.index, len(data.columns)),
    'Ticker': np.tile(data.columns, len(data)),
    'Adjusted Close': data.values.flatten(),
    'Today to Tomorrow': today_to_tomorrow.values.flatten(),
    'Yesterday to Today': yesterday_to_today.values.flatten(),
    'Next Day Close': next_day_adj_close.values.flatten(),
    'Previous Day Close': prev_day_adj_close.values.flatten(),
    'Return': returns.values.flatten(),
    'Volatility': daily_volatility.values.flatten(),
    'RSI': rsi.values.flatten(),
    'SMA_50': sma_50.values.flatten(),
    'SMA_100': sma_100.values.flatten(),
    'SMA_200': sma_200.values.flatten(),
    'Upper Band': upper_band.values.flatten(),
    'Lower Band': lower_band.values.flatten(),
    'Support': support.values.flatten(),
    'Resistance': resistance.values.flatten(),
    'Action': actions.values.flatten(),
},
)

# Replace inf values with NaN using an assignment
indicators_df['Return'] = indicators_df['Return'].replace([np.inf, -np.inf], np.nan)

# Drop NaN values
# indicators_df.dropna(subset=['Adjusted Close', 'Return', 'Volatility', 'RSI', 'SMA_50', 'SMA_100', 'SMA_200', 'Upper Band', 'Lower Band', 'Support', 'Resistance', 'Action'], inplace=True)

# Convert to datetime (Without timezone)
indicators_df['Date'] = pd.to_datetime(indicators_df['Date'], utc=True).dt.date
indicators_df['Date'] = pd.to_datetime(indicators_df['Date'])

# Optimize the DataFrame by converting data types to more memory-efficient types
indicators_df[['Ticker', 'Action']] = indicators_df[['Ticker', 'Action']].astype('category')
float_cols = indicators_df.select_dtypes(include=['float64']).columns
indicators_df[float_cols] = indicators_df[float_cols].astype('float32')

# Display DataFrame
print_title(f"`indicators_df` Shape: {indicators_df.shape}")
display(indicators_df.head())
display(indicators_df.tail())

╔═══════════════════════════════════════════════════════════════╗
║             `indicators_df` Shape: (2246985, 18)              ║
╚═══════════════════════════════════════════════════════════════╝


,Date,Ticker,Adjusted Close,Today to Tomorrow,Yesterday to Today,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
0,2007-01-03,A,21.975,1.000,NaN,22.046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-01-03,AAPL,2.527,1.000,NaN,2.583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-01-03,ABBV,0.000,0.000,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-01-03,ABNB,0.000,0.000,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-01-03,ABT,15.535,1.000,NaN,15.830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Date,Ticker,Adjusted Close,Today to Tomorrow,Yesterday to Today,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
2246980,2024-10-25,XYL,130.420,NaN,-1.000,NaN,130.650,-0.002,0.009,39.080,133.511,134.419,130.849,138.398,129.944,126.710,137.530,NaN
2246981,2024-10-25,YUM,133.040,NaN,-1.000,NaN,133.160,-0.001,0.007,34.396,134.611,133.444,134.008,139.508,130.664,129.710,139.920,NaN
2246982,2024-10-25,ZBH,102.350,NaN,-1.000,NaN,104.000,-0.016,0.011,50.277,107.366,107.970,115.375,107.721,101.313,101.770,115.912,NaN
2246983,2024-10-25,ZBRA,360.090,NaN,-1.000,NaN,362.050,-0.005,0.010,43.894,355.893,336.892,311.042,379.993,359.854,320.770,377.680,NaN
2246984,2024-10-25,ZTS,180.010,NaN,-1.000,NaN,181.500,-0.008,0.013,36.631,189.096,183.316,179.244,197.865,182.303,180.010,196.480,NaN


### Remove data from 2007 after obtaining values

In [31]:
# Remove dates from `2007-01-01` to `2007-12-31`
remove_date_start = "2007-01-01"
remove_date_end = "2007-12-31"

# Filter out the dates to remove
remove_dates_filter = (indicators_df['Date'] >= remove_date_start) & (indicators_df['Date'] <= remove_date_end)

# Remove the dates from the DataFrame
clean_indicators_df = indicators_df.loc[~remove_dates_filter].copy()

# print first date and last date (first and last row date column)
print("First Date:", clean_indicators_df['Date'].iloc[0])
print("Last Date:", clean_indicators_df['Date'].iloc[-1])

First Date: 2008-01-02 00:00:00
Last Date: 2024-10-25 00:00:00


In [32]:
# Export to a new CSV file
file_name = f"updated_w_nas.csv"
file_path = f"data/raw_data/{file_name}"

save_data(clean_indicators_df, file_path)

╔═══════════════════════════════════════════════════════════════╗
║  File `updated_w_nas.zip` already exists. Overwriting file.   ║
╚═══════════════════════════════════════════════════════════════╝
╔═══════════════════════════════════════════════════════════════╗
║         File saved and zipped as `updated_w_nas.zip`          ║
╚═══════════════════════════════════════════════════════════════╝


### Data Visualization
___

In [ ]:
# Plot RSI and Bollinger Bands for a sample stock (e.g., AAPL)
ticker_sample = 'AAPL'
plt.figure(figsize=(14, 7))
plt.subplot(2, 1, 1)
plt.plot(data[ticker_sample], label=f'{ticker_sample} Price')
plt.plot(sma_50[ticker_sample], label='50-Day SMA', linestyle='--')
plt.plot(upper_band[ticker_sample], label='Upper Bollinger Band', linestyle='--', color='orange')
plt.plot(lower_band[ticker_sample], label='Lower Bollinger Band', linestyle='--', color='orange')
plt.fill_between(data.index, lower_band[ticker_sample], upper_band[ticker_sample], color='lightgray')
plt.legend()
plt.title(f'{ticker_sample} Price with Bollinger Bands')
plt.subplot(2, 1, 2)
plt.plot(rsi[ticker_sample], label='RSI', color='purple')
plt.axhline(70, color='red', linestyle='--')
plt.axhline(30, color='green', linestyle='--')
plt.legend()
plt.title(f'{ticker_sample} RSI')
plt.tight_layout()
plt.show()

In [ ]:
# Prepare data for plotting
ticker_sample = 'AAPL'
data_sample = data[[ticker_sample]].copy()
data_sample['SMA_50'] = sma_50[ticker_sample]
data_sample['SMA_100'] = sma_100[ticker_sample]
data_sample['SMA_200'] = sma_200[ticker_sample]
data_sample['RSI'] = rsi[ticker_sample]
data_sample['Support'] = support[ticker_sample]
data_sample['Resistance'] = resistance[ticker_sample]

# Plot
price_plot = data_sample.hvplot.line(y=[ticker_sample, 'SMA_50', 'SMA_100', 'SMA_200'], title=f'{ticker_sample} Price with Bollinger Bands, SMA, and Resistance Levels', ylabel='Price')
support_resistance_plot = data_sample.hvplot.line(y=['Support', 'Resistance'], line_dash='dashed', color=['green', 'red'])
rsi_plot = data_sample.hvplot.line(y='RSI', title=f'{ticker_sample} RSI', ylabel='RSI').opts(yformatter='%.0f') * hv.HLine(70).opts(color='red', line_dash='dashed') * hv.HLine(30).opts(color='green', line_dash='dashed')

(price_plot * support_resistance_plot + rsi_plot).cols(1)


In [ ]:
# %% Plot the trading signals for AAPL

ticker_sample = 'AAPL'

# Extract the price data and actions for AAPL
price_data = data[ticker_sample]
actions_aapl = actions[ticker_sample]

# Define marker styles for each action
marker_styles = {
    'buy': {'marker': '^', 'color': 'green', 'label': 'Buy', 'markersize': 10},
    'sell': {'marker': 'v', 'color': 'red', 'label': 'Sell', 'markersize': 10},
    'hold': {'marker': 'o', 'color': 'blue', 'label': 'Hold', 'markersize': 8},
    'short': {'marker': 'x', 'color': 'purple', 'label': 'Short', 'markersize': 10}
}

# Plot the stock price
plt.figure(figsize=(14, 7))
plt.plot(price_data, label=f'{ticker_sample} Adjusted Close', color='black', lw=2)

# Plot the markers for actions
for action, style in marker_styles.items():
    action_mask = actions_aapl == action
    plt.plot(price_data.index[action_mask], price_data[action_mask], 
             style['marker'], color=style['color'], label=style['label'], markersize=style['markersize'])

# Add titles, labels, and legends
plt.title(f'{ticker_sample} Adjusted Close Price with Trading Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# %% Plot the trading signals for AAPL using hvPlot

ticker_sample = 'AAPL'

# Prepare the price data and actions for AAPL
price_data = data[[ticker_sample]].copy()
price_data['Action'] = actions[ticker_sample]

# Define marker styles for each action
action_colors = {'buy': 'green', 'sell': 'red', 'hold': 'blue', 'short': 'purple'}

# Create a base line plot for the adjusted close price
price_plot = price_data.hvplot.line(
    y=ticker_sample, 
    title=f'{ticker_sample} Adjusted Close Price with Trading Signals',
    ylabel='Price', 
    line_width=2,
    height=400,
    width=800,
    legend=False
)

# Overlay the action markers
buy_markers = price_data[price_data['Action'] == 'buy'].hvplot.scatter(
    y=ticker_sample, marker='^', color=action_colors['buy'], size=10, label='Buy'
)
sell_markers = price_data[price_data['Action'] == 'sell'].hvplot.scatter(
    y=ticker_sample, marker='v', color=action_colors['sell'], size=10, label='Sell'
)
hold_markers = price_data[price_data['Action'] == 'hold'].hvplot.scatter(
    y=ticker_sample, marker='o', color=action_colors['hold'], size=6, label='Hold'
)
short_markers = price_data[price_data['Action'] == 'short'].hvplot.scatter(
    y=ticker_sample, marker='x', color=action_colors['short'], size=10, label='Short'
)

# Combine the price plot and the markers
interactive_plot = price_plot * buy_markers * sell_markers * hold_markers * short_markers

# Display the interactive plot with zoom functionality
interactive_plot.opts(legend_position='top_left')
interactive_plot
